In [1]:
debugging = True

# Description

This notebook allows to combine (**add** or **mean**) images from different folders


**INPUT**

```
folder_1
   image001.fits
   image002.fits
   image003.fits
   ....
   
folder_2
    image001.fits
    image002.fits
    image003.fits
    ...
    
...    
    
```
**OUTPUT**

```
folder_combine
    image001.fits
    image002.fits
    image003.fits
    ...
```



# Python Import

In [2]:
from __code.__all import custom_style
custom_style.style()

In [3]:
from ipywidgets.widgets import interact
from ipywidgets import widgets
from IPython.core.display import display, HTML

import glob
import pprint

from __code.merge_folders import fileselector, utilities, file_handler

ipts = utilities.get_ipts()
working_dir = utilities.get_working_dir(ipts=ipts)
if debugging:
    working_dir = '/Volumes/my_book_thunderbolt_duo/IPTS/IPTS_19037'


# Folders to Merge 

## Select Folders 

In [5]:
folder_list_widget = fileselector.FileSelectorPanel(instruction='select folder to combine', 
                                     start_dir=working_dir, 
                                     type='directory', 
                                     multiple=True)
folder_list_widget.show()

# Merging Method

## How many folders do you want to combine together

In [6]:
nbr_folder = len(folder_list_widget.result())
radio_list_string = [str(_index) for _index in np.arange(2, nbr_folder+1)]
bin_size = widgets.RadioButtons(options=radio_list_string,
                               value=radio_list_string[0])
display(bin_size)

##  How do you want to combine them

In [7]:
combine_method = widgets.RadioButtons(options=['add','mean'],
                                             value='add')
display(combine_method)

# Merging 

Checking that number of files in each folder is identical

In [8]:
# retrieving name of images in each file
list_folder = folder_list_widget.result()

# initialization of dictionary that will store the list of files
list_files_dict = {}


full_folder_path = [os.path.join(folder_list_widget.curdir, _folder) for _folder in list_folder]

short_and_long_folder_name = zip(list_folder, full_folder_path)
nbr_files = {}
for _short, _long in short_and_long_folder_name:
    _list_files = glob.glob(_long + "/*.*")
    list_files_dict[_short] = _list_files
    nbr_files[_short] = len(_list_files)

# checking the folders have the same number of files
values = set(nbr_files.values())
if len(values) > 1:
    raise ValueError("Folder do not have the same number of files!")

In [11]:
def load_list_files(filename_array=[]):
    data = []
    for _filename in filename_array:
        _data = file_handler.load_data(filename=_filename)
        data.append(_data)
    return data

def add(array=[]):
    return np.array(array).sum(axis=0)

def mean(array=[]):
    return np.array(array).mean(axis=0)

In [13]:
merged_images = {'file_name': [],
                'data': []}
nbr_files = list(values)[0]

box1 = widgets.HBox([widgets.Label("Merging Progress:",
                                      layout=widgets.Layout(width='10%')),
                        widgets.IntProgress(max=nbr_files)])
display(box1)
w1 = box1.children[1]

for _index_file in np.arange(nbr_files):
    _list_file_to_merge = []
    for _key in list_files_dict.keys():
        _file = list_files_dict[_key][_index_file]
        _list_file_to_merge.append(_file)
        merged_images['file_name'].append(os.path.basename(_file))
    
    _data_array = load_list_files(_list_file_to_merge)
    merged_data = add(_data_array)
    
    w1.value = _index_file + 1
    
    

['/Volumes/my_book_thunderbolt_duo/IPTS/IPTS_19037/deal_images/Image001/image_SummedImg.fits',
 '/Volumes/my_book_thunderbolt_duo/IPTS/IPTS_19037/deal_images/Image002/image_SummedImg.fits',
 '/Volumes/my_book_thunderbolt_duo/IPTS/IPTS_19037/deal_images/Image003/image_SummedImg.fits',
 '/Volumes/my_book_thunderbolt_duo/IPTS/IPTS_19037/deal_images/Image004/image_SummedImg.fits']

In [3]:
verbose = False
debugging = True

try:
    from PyQt4.QtGui import QFileDialog
    from PyQt4 import QtCore, QtGui
    from PyQt4.QtGui import QMainWindow
except ImportError:
    from PyQt5.QtWidgets import QFileDialog
    from PyQt5 import QtCore, QtGui
    from PyQt5.QtWidgets import QApplication
    
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib notebook

from collections import OrderedDict

from ipywidgets.widgets import interact
from ipywidgets import widgets
from IPython.core.display import display, HTML

from pprint import pprint

import datetime
import os
import glob
import shutil
import re
import scipy.ndimage

import pyqtgraph as pg

from IPython.display import display
from IPython import display as display_ipython

#from __code import file_handler
from __code import fileselector
from __code import utilities
#from __code import gui_widgets

ipts = utilities.get_ipts()
working_dir = utilities.get_working_dir(ipts=ipts)
if debugging:
    working_dir = '/Volumes/my_book_thunderbolt_duo/IPTS/IPTS_19037'

In [4]:
folder_list_widget = fileselector.FileSelectorPanel(instruction='select folder to combine', 
                                     start_dir=working_dir, 
                                     type='directory', 
                                     multiple=True)
folder_list_widget.show()

# Combine fits files with same name from n different folders 

#### Algorithms available
 * Add
 * Mean

In [1]:
import os

from IPython.html import widgets
from IPython.display import display

import matplotlib.pyplot as plt

import glob
import pyfits

import widgets_handler

/Users/j35/anaconda/envs/py35/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
root_folder =  "/Volumes/

### Select folder n

#### Select all the folders to add... click CANCEL to let the program know you do not have any more folder to add 

In [5]:
%gui qt

In [6]:
list_folders = []

while(True):
    folder_n = widgets_handler.gui_fname(dir = root_folder)
    if folder_n is '':
        break
    list_folders.append(folder_n)
%matplotlib inline

### Check that all folders have the same number of images

In [7]:
number_of_files = []
list_files_in_folders = []
for _folder in list_folders:
    list_fname = glob.glob(_folder + '/*.fits')
    number_of_files.append(len(list_fname))
    list_files_in_folders.append(list_fname)

assert len(set(number_of_files)) == 1    

## Loading the data 

In [8]:
w = widgets.IntProgress()
w.max = len(list_folders)-1
display(w)

index = 0
data_folders = []

for index_folder, list_folder in enumerate(list_files_in_folders):

    _data_folder = []
    for index_file, filename in enumerate(list_folder):
        hdu_list = pyfits.open(filename)
        hdu = hdu_list[0]
        _data_folder.append(hdu.data)
    
    data_folders.append(_data_folder)
    w.value = index_folder

In [9]:
import numpy as np
np.shape(data_folders)

(3, 1485, 512, 512)

## Retrieving images index

In [10]:
def digit_file_name(full_file_name):
    base_name = os.path.basename(full_file_name)
    [short_name, ext] = base_name.split('.')
    [image_name, image_sequence_number] = short_name.split('_')
    image_name_split = image_name.split('Image')
    image_number = image_name_split[1]
    return [image_name, image_number, image_sequence_number]

In [11]:
list_image_numbers = []
for index_folder, list_folder in enumerate(list_files_in_folders):
    _first_image = list_folder[0]
    [image_name, image_number, image_sequence_number] = digit_file_name(_first_image)
    list_image_numbers.append(image_number)
    
list_image_sequence = []
for index_file, _file in enumerate(list_files_in_folders[0]):
    [image_name, image_number, image_sequence_number] = digit_file_name(_file)
    list_image_sequence.append(image_sequence_number)
    

## Add data 

In [12]:
w = widgets.IntProgress()
number_of_files = len(data_folders[0])
w.max = number_of_files -1
display(w)

sum_data = []
sum_names = []
for index_file in range(number_of_files):

    _add_data = []
    tmp_name = ''
    for index_folder, folder_name in enumerate(list_files_in_folders):
        _list_data = data_folders[index_folder]
        _data = _list_data[index_file]
        [image_name, image_number, image_seqence_number] = digit_file_name(folder_name[index_file])
        if index_folder == 0:
            _add_data = _data
            _tmp_name = "%.2d" %int(image_number)
        else:
            _add_data += _data
            _tmp_name += '_%.2d' %int(image_number)
    
    sum_data.append(_add_data)
    sum_names.append(_tmp_name)
    w.value = index_file

## Exporting data 

In [13]:
from PyQt4 import QtCore, QtGui

def gui_fname(dir=None):
    """Select a folder
    """
    if dir is None: dir ='./'
    dir_name = QtGui.QFileDialog.getExistingDirectory(None, "Select Folder ...",
                                                  dir,
                                                  QtGui.QFileDialog.ShowDirsOnly)
    return dir_name

In [14]:
%gui qt
dir_name = gui_fname(dir = root_folder)
%matplotlib inline

In [15]:
w = widgets.IntProgress()
w.max = len(sum_data)-1
display(w)

for index in range(len(sum_data)):
    
    full_name = os.path.join(dir_name, sum_names[index] + '_' + list_image_sequence[index] + '.fits')

    hdu = pyfits.PrimaryHDU(sum_data[index])
    hdulist = pyfits.HDUList([hdu])
    hdulist.writeto(full_name)
    
    w.value = index

# References

